<img src = "https://i.imgur.com/UjutVJd.jpg" align = "center">


# Text generation using RNN/LSTM (Character-level)
In this notebook you will learn the How to use TensorFlow for create a Recurrent Neural Network<br />

# Table of contents

<div>
- <a href="#intro">Introduction</a><br />
- <a href="#arch">Architectures</a><br />
- <a href="#lstm">Long Short-Term Memory Model (LSTM)</a><br />
- <a href="#build">Building a LSTM with TensorFlow</a>
</div>

<hr>

This code implements a Recurrent Neural Network with LSTM/RNN units for training/sampling from character-level language models. In other words, the model takes a text file as input and trains the RNN network that learns to predict the next character in a sequence.  
The RNN can then be used to generate text character by character that will look like the original training data. 

This code is based on this [blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), and the code is an step-by-step implimentation of the [character-level implimentation](https://github.com/crazydonkey200/tensorflow-char-rnn).




First, import the requiered libraries:

In [0]:
import tensorflow as tf
import time
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np
#from tensorflow.python.ops import rnn_cell
#from tensorflow.python.ops import seq2seq

In [2]:
from google.colab import files
uploaded = files.upload()

Saving indo.txt to indo.txt


### Data loader
The following cell is a class that help to read data from input file.

In [0]:
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "indo.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
            data = f.read()
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        # When the data (tensor) is too small, let's give them a better error message
        if self.num_batches==0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)


    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

### Parameters
#### Batch, number_of_batch, batch_size and seq_length
what is batch, number_of_batch, batch_size and seq_length in the charcter level example?  

Lets assume the input is this sentence: '__here is an example__'. Then:
- txt_length = 18  
- seq_length = 3  
- batch_size = 2  
- number_of_batchs = 18/3*2 = 3
- batch = array (['h','e','r'],['e',' ','i'])
- sample Seq = 'her'  

Ok, now, lets look at a real dataset, with real parameters. 

In [0]:
seq_length = 50 # RNN sequence length
batch_size = 60  # minibatch size, i.e. size of data in each epoch
num_epochs = 125 # you should change it to 50 if you want to see a relatively good results
learning_rate = 0.002
decay_rate = 0.97
rnn_size = 128 # size of RNN hidden state (output dimension)
num_layers = 2 #number of layers in the RNN

We download the input file, and print a part of it:

In [5]:
from google.colab import files
with open('indo.txt', 'r') as f:
    read_data = f.read()
    print read_data[0:10]
f.closed

﻿
ya say


True

Now, we can read the data at batches using the __TextLoader__ class. It will convert the characters to numbers, and represent each sequence as a vector in batches:

In [6]:
data_loader = TextLoader('', batch_size, seq_length)
vocab_size = data_loader.vocab_size
print "vocabulary size:" ,data_loader.vocab_size
print "Characte8rs:" ,data_loader.chars
print "vocab number of 'F':",data_loader.vocab['F']
print "Character sequences (first batch):", data_loader.x_batches[0]

reading text file
vocabulary size: 299
Characte8rs: (u'a', u' ', u'n', u'i', u'e', u'k', u't', u'u', u'm', u'r', u's', u'd', u'g', u'l', u'p', u'b', u'h', u'y', u'o', u'\n', u'.', u'j', u'c', u',', u'!', u'A', u'w', u'f', u'-', u'?', u'"', u'S', u':', u'v', u'0', u'1', u';', u'&', u'2', u')', u'/', u'4', u'(', u'z', u'5', u'3', u'x', u'\U0001f602', u'T', u"'", u'\ufe0f', u'7', u'9', u'8', u'6', u'K', u'*', u'\u2764', u'M', u'B', u'$', u'P', u'J', u'%', u'H', u'+', u'\U0001f62d', u'q', u'C', u'\u200b', u'D', u'N', u'I', u'Y', u'G', u'\U0001f64f', u'O', u'\U0001f60a', u'\U0001f3fc', u'\U0001f643', u'\U0001f44d', u'\U0001f644', u'\U0001f601', u'W', u'@', u'R', u'\U0001f3fb', u'\U0001f618', u'\U0001f4af', u'L', u'\U0001f44f', u'\U0001f3fd', u'^', u'=', u'\u2026', u'~', u'\U0001f629', u'\u2014', u'\U0001f64c', u'\U0001f60d', u'\U0001f497', u'\u2728', u'F', u'V', u'\U0001f61c', u'\U0001f51c', u'\U0001f525', u'\U0001f1fa', u'\U0001f914', u'\U0001f918', u'\u2b50', u'\U0001f440', u'\U0001f609',

### Input and output

In [7]:
x,y = data_loader.next_batch()
x

array([[230,  19,  17, ...,   6,   7,  16],
       [  7,   5,   1, ...,   6,   4,   2],
       [ 14,   9,   3, ...,  15,   0,  12],
       ...,
       [  1,  16,   3, ...,   4,   8,   0],
       [  7,   1,   3, ...,   5,   1,   8],
       [ 16,   1,  14, ...,   8,   0,  13]])

In [8]:
x.shape  #batch_size =60, seq_length=50

(60, 50)

Here, __y__ is the next character for each character in __x__:

In [9]:
y

array([[19, 17,  0, ...,  7, 16,  5],
       [ 5,  1,  6, ...,  4,  2,  6],
       [ 9,  3, 15, ...,  0, 12,  7],
       ...,
       [16,  3, 11, ...,  8,  0, 16],
       [ 1,  3,  6, ...,  1,  8,  4],
       [ 1, 14,  0, ...,  0, 13,  0]])

### LSTM Architecture
Each LSTM cell has 5 parts:
1. Input
2. prv_state
3. prv_output
4. new_state
5. new_output


- Each LSTM cell has an input layre, which its size is 128 units in our case. The input vector's dimension also is 128, which is the dimensionality of embedding vector, so called, dimension size of W2V/embedding, for each character/word.
- Each LSTM cell has a hidden layer, where there are some hidden units. The argument n_hidden=128 of BasicLSTMCell is the number of hidden units of the LSTM (inside A). It keeps the size of the output and state vector. It is also known as, rnn_size, num_units, num_hidden_units, and LSTM size
- An LSTM keeps two pieces of information as it propagates through time: 
    - __hidden state__ vector: Each LSTM cell accept a vector, called __hidden state__ vector, of size n_hidden=128, and its value is returned to the LSTM cell in the next step. The __hidden state__ vector; which is the memory of the LSTM, accumulates using its (forget, input, and output) gates through time. "num_units" is equivalant to "size of RNN hidden state". number of hidden units is the dimensianality of the output (= dimesianality of the state) of the LSTM cell.
    - __previous time-step output__: For each LSTM cell that we initialize, we need to supply a value (128 in this case) for the hidden dimension, or as some people like to call it, the number of units in the LSTM cell. 


#### num_layers = 2 
- number of layers in the RNN, is defined by num_layers
- An input of MultiRNNCell is __cells__ which is list of RNNCells that will be composed in this order.

### Defining stacked RNN Cell

__BasicRNNCell__ is the most basic RNN cell.

In [10]:
cell = tf.contrib.rnn.BasicRNNCell(rnn_size)

W0810 18:10:31.169241 140694890518400 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0810 18:10:31.171355 140694890518400 deprecation.py:323] From <ipython-input-10-cbc7d8d66937>:1: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [11]:
# a two layer cell
stacked_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)

W0810 18:10:31.196155 140694890518400 deprecation.py:323] From <ipython-input-11-32025279f672>:1: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0810 18:10:31.198925 140694890518400 rnn_cell_impl.py:1642] At least two cells provided to MultiRNNCell are the same object and will share weights.


In [12]:
# hidden state size
stacked_cell.output_size

128

__state__ varibale keeps output and new_state of the LSTM, so it is a touple of size:

In [13]:
stacked_cell.state_size

(128, 128)

Lets define input data:

In [14]:
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])# a 60x50
input_data

<tf.Tensor 'Placeholder:0' shape=(60, 50) dtype=int32>

and target data:

In [15]:
targets = tf.placeholder(tf.int32, [batch_size, seq_length]) # a 60x50
targets

<tf.Tensor 'Placeholder_1:0' shape=(60, 50) dtype=int32>

The memory state of the network is initialized with a vector of zeros and gets updated after reading each character.

__BasicRNNCell.zero_state(batch_size, dtype)__ Return zero-filled state tensor(s). In this function, batch_size
representing the batch size.

In [0]:
initial_state = stacked_cell.zero_state(batch_size, tf.float32) #why batch_size ? 60x128

Lets check the value of the input_data again:

In [17]:
session = tf.Session()
feed_dict={input_data:x, targets:y}
session.run(input_data, feed_dict)

array([[230,  19,  17, ...,   6,   7,  16],
       [  7,   5,   1, ...,   6,   4,   2],
       [ 14,   9,   3, ...,  15,   0,  12],
       ...,
       [  1,  16,   3, ...,   4,   8,   0],
       [  7,   1,   3, ...,   5,   1,   8],
       [ 16,   1,  14, ...,   8,   0,  13]], dtype=int32)

### Embedding
In this section, we build a 128-dim vector for each character. As we have 60 batches, and 50 character in each sequence, it will generate a [60,50,128] matrix.

__Notice:__ The function `tf.get_variable()` is used to share a variable and to initialize it in one place. `tf.get_variable()` is used to get or create a variable instead of a direct call to `tf.Variable`. 

In [18]:
with tf.variable_scope('rnnlm', reuse=False):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
    softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65)
    #with tf.device("/cpu:0"):
        
    # embedding variable is initialized randomely
    embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128

    # embedding_lookup goes to each row of input_data, and for each character in the row, finds the correspond vector in embedding
    # it creates a 60*50*[1*128] matrix
    # so, the first elemnt of em, is a matrix of 50x128, which each row of it is vector representing that character
    em = tf.nn.embedding_lookup(embedding, input_data) # em is 60x50x[1*128]
    # split: Splits a tensor into sub tensors.
    # syntax:  tf.split(split_dim, num_split, value, name='split')
    # it will split the 60x50x[1x128] matrix into 50 matrix of 60x[1*128]
    inputs = tf.split(em, seq_length, 1)
    # It will convert the list to 50 matrix of [60x128]
    inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

W0810 18:10:31.431709 140694890518400 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Lets take a look at the __embedding__, __em__, and __inputs__ variabbles:

Embedding variable is initialized with random values:

In [19]:
session.run(tf.global_variables_initializer())
#print embedding.shape
session.run(embedding)

array([[-0.10144918, -0.01748811, -0.11183719, ..., -0.07099854,
        -0.08114132,  0.11085897],
       [ 0.08256572, -0.05597642, -0.0846906 , ..., -0.04284342,
        -0.07012019,  0.03928942],
       [ 0.0630882 , -0.08845006, -0.08097921, ...,  0.04365886,
        -0.01455108, -0.06951474],
       ...,
       [-0.03949924, -0.05135806,  0.0248125 , ...,  0.02303206,
        -0.07098458,  0.10671761],
       [-0.09509382,  0.0817465 , -0.00462659, ..., -0.06856398,
         0.07000429, -0.01556246],
       [-0.01298703, -0.05726963, -0.05212588, ..., -0.003806  ,
        -0.05807538,  0.09797091]], dtype=float32)

The first elemnt of em, is a matrix of 50x128, which each row of it is vector representing that character

In [20]:
em = tf.nn.embedding_lookup(embedding, input_data)
emp = session.run(em,feed_dict={input_data:x})
print emp.shape
emp[0]

(60, 50, 128)


array([[ 0.00193801,  0.06445912, -0.01829027, ...,  0.01161655,
        -0.08406152,  0.02663159],
       [-0.02413026, -0.05376114, -0.01900711, ..., -0.09488733,
        -0.10255954,  0.03039183],
       [-0.11780012, -0.00911416, -0.02419504, ...,  0.05783693,
         0.09116145, -0.04370993],
       ...,
       [-0.03913486, -0.01289552, -0.06907159, ...,  0.11151674,
        -0.07500167, -0.10681166],
       [ 0.06671915,  0.01879836,  0.08078011, ...,  0.0150932 ,
        -0.04580459,  0.05674743],
       [ 0.04672904, -0.04739361,  0.04124486, ...,  0.09046816,
         0.0047625 ,  0.02865802]], dtype=float32)

Let's consider each sequence as a sentence of length 50 characters, then, the first item in __inputs__ is a [60x128] vector which represents the first characters of 60 sentences.

In [21]:
inputs = tf.split(em, seq_length, 1)
inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
inputs[0:5]

[<tf.Tensor 'Squeeze:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_2:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_4:0' shape=(60, 128) dtype=float32>]

### Feeding a batch of 50 sequence to a RNN:

The feeding process for iputs is as following:

- Step 1:  first character of each of the 50 sentences (in a batch) is entered in parallel.  
- Step 2:  second character of each of the 50 sentences is input in parallel. 
- Step n: nth character of each of the 50 sentences is input in parallel.  

The parallelism is only for efficiency.  Each character in a batch is handled in parallel,  but the network sees one character of a sequence at a time and does the computations accordingly. All the computations involving the characters of all sequences in a batch at a given time step are done in parallel. 

In [22]:
session.run(inputs[0],feed_dict={input_data:x})

array([[ 0.00193801,  0.06445912, -0.01829027, ...,  0.01161655,
        -0.08406152,  0.02663159],
       [ 0.06671915,  0.01879836,  0.08078011, ...,  0.0150932 ,
        -0.04580459,  0.05674743],
       [ 0.05471622, -0.0527781 ,  0.03584961, ..., -0.10134176,
         0.02348984, -0.01711076],
       ...,
       [ 0.08256572, -0.05597642, -0.0846906 , ..., -0.04284342,
        -0.07012019,  0.03928942],
       [ 0.06671915,  0.01879836,  0.08078011, ...,  0.0150932 ,
        -0.04580459,  0.05674743],
       [ 0.04672904, -0.04739361,  0.04124486, ...,  0.09046816,
         0.0047625 ,  0.02865802]], dtype=float32)

Feeding the RNN with one batch, we can check the new output and new state of network:

In [23]:
#outputs is 50x[60*128]
outputs, new_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state, stacked_cell, loop_function=None, scope='rnnlm')
new_state

W0810 18:10:32.400255 140694890518400 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/rnn_cell_impl.py:459: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


(<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_98:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_99:0' shape=(60, 128) dtype=float32>)

In [24]:
outputs[0:5]

[<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_5:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_7:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_9:0' shape=(60, 128) dtype=float32>]

Let's check the output of network after feeding it with first batch:

In [25]:
first_output = outputs[0]
session.run(tf.global_variables_initializer())
session.run(first_output,feed_dict={input_data:x})

array([[ 0.0046565 , -0.00428928, -0.03925613, ..., -0.00226973,
        -0.09883209, -0.01093096],
       [-0.02663217, -0.01438374, -0.06386439, ...,  0.07337715,
         0.02132489,  0.0110517 ],
       [-0.00822803,  0.0165975 ,  0.05339247, ...,  0.01013395,
         0.06190556,  0.05736312],
       ...,
       [-0.03445358,  0.0544724 , -0.00655166, ..., -0.01424177,
         0.07914845, -0.00403997],
       [-0.02663217, -0.01438374, -0.06386439, ...,  0.07337715,
         0.02132489,  0.0110517 ],
       [-0.00748037, -0.06851382, -0.0138286 , ...,  0.0218    ,
        -0.01719994, -0.0102007 ]], dtype=float32)

As it was explained, __outputs__ variable is a 50x[60x128] tensor. We need to reshape it back to [60x50x128] to be able to calculate the probablity of the next character using the softmax. The __softmax_w__ shape is [rnn_size, vocab_size],whihc is [128x65] in our case. Threfore, we have a fully connected layer on top of LSTM cells, which help us to decode the next charachter. We can use the __softmax(output * softmax_w + softmax_b)__ for this purpose. The shape of the matrixis would be:

softmax([60x50x128]x[128x65]+[1x65]) = [60x50x65]

We can do it step-by-step:

In [26]:
output = tf.reshape(tf.concat( outputs,1), [-1, rnn_size])
output

<tf.Tensor 'Reshape:0' shape=(3000, 128) dtype=float32>

In [27]:
logits = tf.matmul(output, softmax_w) + softmax_b
logits

<tf.Tensor 'add:0' shape=(3000, 299) dtype=float32>

In [28]:
probs = tf.nn.softmax(logits)
probs

<tf.Tensor 'Softmax:0' shape=(3000, 299) dtype=float32>

Here is the probablity of the next chracter in all batches:

In [29]:
session.run(tf.global_variables_initializer())
session.run(probs,feed_dict={input_data:x})

array([[0.00364071, 0.0033711 , 0.00368637, ..., 0.003285  , 0.00308801,
        0.00365575],
       [0.00358343, 0.00322644, 0.00369695, ..., 0.00280995, 0.00346353,
        0.00340522],
       [0.00307891, 0.00323082, 0.00348126, ..., 0.00315973, 0.00300752,
        0.00351491],
       ...,
       [0.00311944, 0.00341824, 0.0032606 , ..., 0.00288109, 0.00359647,
        0.00370034],
       [0.00342626, 0.00337893, 0.00396319, ..., 0.0026006 , 0.00373331,
        0.00348507],
       [0.00338898, 0.00367759, 0.00430869, ..., 0.00292734, 0.0030666 ,
        0.00441798]], dtype=float32)

Now, we are in the position to calculate the cost of training with __loss function__, and keep feedng the network to learn it. But, the question is: what the LSTM networks learn?

In [30]:
grad_clip =5.
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'rnnlm/softmax_w:0' shape=(128, 299) dtype=float32_ref>,
 <tf.Variable 'rnnlm/softmax_b:0' shape=(299,) dtype=float32_ref>,
 <tf.Variable 'rnnlm/embedding:0' shape=(299, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/bias:0' shape=(128,) dtype=float32_ref>]

# All together
Now, let's put all of parts together in a class, and train the model:

In [0]:
class LSTMModel():
    def __init__(self,sample=False):
        rnn_size = 128 # size of RNN hidden state vector
        batch_size = 60 # minibatch size, i.e. size of dataset in each epoch
        seq_length = 50 # RNN sequence length
        num_layers = 2 # number of layers in the RNN
        vocab_size = 65
        grad_clip = 5.
        if sample:
            print(">> sample mode:")
            batch_size = 1
            seq_length = 1
        # The core of the model consists of an LSTM cell that processes one char at a time and computes probabilities of the possible continuations of the char. 
        basic_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
        # model.cell.state_size is (128, 128)
        self.stacked_cell = tf.contrib.rnn.MultiRNNCell([basic_cell] * num_layers)

        self.input_data = tf.placeholder(tf.int32, [batch_size, seq_length], name="input_data")
        self.targets = tf.placeholder(tf.int32, [batch_size, seq_length], name="targets")
        # Initial state of the LSTM memory.
        # The memory state of the network is initialized with a vector of zeros and gets updated after reading each char. 
        self.initial_state = stacked_cell.zero_state(batch_size, tf.float32) #why batch_size

        with tf.variable_scope('rnnlm_class1'):
            softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
            softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65
            with tf.device("/cpu:0"):
                embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128
                inputs = tf.split(tf.nn.embedding_lookup(embedding, self.input_data), seq_length, 1)
                inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
                #inputs = tf.split(em, seq_length, 1)
                
                


        # The value of state is updated after processing each batch of chars.
        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, self.initial_state, self.stacked_cell, loop_function=None, scope='rnnlm_class1')
        output = tf.reshape(tf.concat(outputs,1), [-1, rnn_size])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([self.logits],
                [tf.reshape(self.targets, [-1])],
                [tf.ones([batch_size * seq_length])],
                vocab_size)
        self.cost = tf.reduce_sum(loss) / batch_size / seq_length
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),grad_clip)
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    
    def sample(self, sess, chars, vocab, num=200, prime='adalah ', sampling_type=1):
        state = sess.run(self.stacked_cell.zero_state(1, tf.float32))
        #print state
        for char in prime[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return(int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        for n in range(num):
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [probs, state] = sess.run([self.probs, self.final_state], feed)
            p = probs[0]

            if sampling_type == 0:
                sample = np.argmax(p)
            elif sampling_type == 2:
                if char == ' ':
                    sample = weighted_pick(p)
                else:
                    sample = np.argmax(p)
            else: # sampling_type == 1 default:
                sample = weighted_pick(p)

            pred = chars[sample]
            ret += pred
            char = pred
        return ret

### Creating the LSTM object
Now we create a LSTM model:

In [32]:
with tf.variable_scope("rnn"):
    model = LSTMModel()

W0810 18:10:35.658432 140694890518400 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/clip_ops.py:286: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Train usinng LSTMModel class
We can train our model through feeding batches:

In [33]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(num_epochs): # num_epochs is 5 for test, but should be higher
        sess.run(tf.assign(model.lr, learning_rate * (decay_rate ** e)))
        data_loader.reset_batch_pointer()
        state = sess.run(model.initial_state) # (2x[60x128])
        for b in range(data_loader.num_batches): #for each batch
            start = time.time()
            x, y = data_loader.next_batch()
            feed = {model.input_data: x, model.targets: y, model.initial_state:state}
            end = time.time()
        print("{}/{} (epoch {}), time/batch = {:.3f}" \
                .format(e * data_loader.num_batches + b, num_epochs * data_loader.num_batches, e, end - start))
        with tf.variable_scope("rnn", reuse=True):
            sample_model = LSTMModel(sample=True)
            print sample_model.sample(sess, data_loader.chars , data_loader.vocab, num=50, prime='Saya ', sampling_type=1)
            print '----------------------------------'

115/14500 (epoch 0), time/batch = 0.000
>> sample mode:
Saya l96))sba2️dPJsj
0 ?814d3h5!i0il)K"jKSg*n&1Kni$3nyS
----------------------------------
231/14500 (epoch 1), time/batch = 0.000
>> sample mode:
Saya 9H6rjnnvTba4️vPa$8p46l yS.r&no$o&b*g%s❤hJ❤m'$)%"K8
----------------------------------
347/14500 (epoch 2), time/batch = 0.000
>> sample mode:
Saya .&M️:t-s65K❤yth&a4?2r$r!%%shoT❤S(T2😂2BlvT)e/Shjdvr
----------------------------------
463/14500 (epoch 3), time/batch = 0.000
>> sample mode:
Saya g&&28dbjPvuSfn*T😂h:%M"4sbon(9Mob,f0❤t3A'u*"wk/egT%
----------------------------------
579/14500 (epoch 4), time/batch = 0.000
>> sample mode:
Saya 4ka*&/️hTgy3!B92B?6!sKfoM?dAr7"b!ym*0.Tna84/JntT7%
----------------------------------
695/14500 (epoch 5), time/batch = 0.000
>> sample mode:
Saya (e()w0"-hf7f%aMo*9mxbo"KdmaKalvu6
e5Kp?uPzkhSc-awt
----------------------------------
811/14500 (epoch 6), time/batch = 0.000
>> sample mode:
Saya !kg31mcal2w.*c
y)%tTu*😂-?sSJb;1H4?j!53j%3a&2u14"/😂
------

# Thanks for completing this lesson!
Created by: <a href = "https://linkedin.com/in/saeedaghabozorgi"> Saeed Aghabozorgi </a></h4>
This code is based on this [blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), and the code is an step-by-step implimentation of the [character-level implimentation](https://github.com/crazydonkey200/tensorflow-char-rnn).

<hr>

<p>Copyright &copy; 2017 IBM <a href="https://cognitiveclass.ai/?utm_source=ML0151&utm_medium=lab&utm_campaign=cclab">IBM Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://cognitiveclass.ai/mit-license/">MIT License</a>.</p>